In [1]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
import acquire
import prepare
import model
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from io import StringIO
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier

# Exercise

### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [ ]:
#acquire data
df = acquire.get_titanic_data()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#split data
train, validate, test = prepare.prep_titanic_data(df, column = 'age', method = 'median', dummies = ['embarked', 'sex'])


In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.head(1)

In [ ]:
# drop out non-numerical columns or non-encoded version remaining in this data set.
#in this case, it is only 'passaner_id'
train.drop(columns = ['passenger_id'], inplace=True)
validate.drop(columns = ['passenger_id'], inplace=True)
test.drop(columns = ['passenger_id'], inplace=True)

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.head(2)

In [ ]:
# We will be attempting to make a Decision Tree Classifier Model that will predict survival on the 
# Titanic that performs better than the baseline

In [ ]:
# we are working with survided. The most prevelant class in the training dataset (the mode)
train.survived.value_counts()

In [ ]:
train.survived.mode()

In [ ]:
#so the baselined is survived = 0
train['baseline'] = 0

In [ ]:
#accuracy
(train.baseline == train.survived).mean()

In [ ]:
print(f'The baseline accuracy for nonsurvival in all cases on the Titanic Dataset is {(train.baseline == train.survived).mean():.3}')

In [ ]:
# classification report of baseline model
baseline_class_report = classification_report(train.survived, train.baseline, zero_division=True)
print(baseline_class_report)

In [ ]:
# #other way to check the accuracy of baseline
# #split 
# X_train, y_train = train.drop(columns='survived'), train.survived
# #as the baseline mode is = 0 , thats why we put constant = 0
# # 1. Create the object
# baseline = DummyClassifier(strategy='constant', constant=0)
# # 2. Fit the object
# baseline.fit(X_train, y_train)

In [ ]:
# print(f'Baseline accuracy: %.4f' % baseline.score(X_train, y_train))

### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [ ]:
train.head(1)

In [ ]:
# remove baseline assumption from the train, we wont need this column anymore
train.drop(columns='baseline', inplace=True)

In [ ]:
train.head()

In [ ]:
# use the funnction that I create:

# X_train, y_train = model.split_Xy(df = train, column = 'survived')
# X_validate, y_validate = model.split_Xy(validate, 'survived') 
# X_test, y_test = model.split_Xy(test, 'survived') 


In [ ]:
# split our X and y
# do the capital X, lowercase y thing for train test and split
# X is the data frame of the features, y is a series of the target
X_train = train.drop(columns='survived')
y_train = train[['survived']]
X_validate, y_validate = validate.drop(columns='survived'), validate['survived']
X_test, y_test = test.drop(columns='survived'), test['survived']

In [ ]:
# #other way to do the split is decide wich columns we are going to use
# X_cols = [ 'pclass','age', 'sex_male','alone', 'embarked_Q', 'embarked_S']
# y_col = 'survived'

# X_train, y_train = train[X_cols], train[y_col]
# X_train, y_train = train[X_cols], train[y_col]
# X_validate, y_validate = validate[X_cols], validate[y_col]
# X_test, y_test = test[X_cols], test[y_col]

In [ ]:
X_train.head(1)

In [ ]:
y_train.head(1)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
# create the model
model = DecisionTreeClassifier()
# fit the model
model.fit(X_train, y_train)

In [ ]:
#calculate the accuracy
acc = model.score(X_train, y_train)
acc

In [ ]:

# look at the model scores for training set and validate set
print(f'training score: {model.score(X_train, y_train):.2%}')
print(f'validate score: {model.score(X_validate, y_validate):.2%}')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
#as the difference between train and validate is big. I will use  max_features and max_depth
#when score.train > score.validate means overfit

In [ ]:
# create the model
model1 = DecisionTreeClassifier(max_depth=3, max_features=3)
# fit the model
model1.fit(X_train, y_train)

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {model1.score(X_train, y_train):.2%}')
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model1,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
#prediction based on our model1
prediction = model1.predict(X_train)

In [ ]:
prediction

In [ ]:
pd.Series(prediction).value_counts()

In [ ]:
#confusion matrix
conf_m = confusion_matrix(y_train, prediction )
conf_m

In [ ]:
#check if rows or columns have the actual number. in this case rows has the actual number
train.survived.value_counts()

In [ ]:
 #just to see
mat =  pd.DataFrame ((confusion_matrix(y_train, prediction )),index = ['actual_dead','actual_survived'], columns =['pred_dead','pred_survived' ])
mat

In [ ]:
rubric_df = pd.DataFrame([['True Negative', 'False positive'], ['False Negative', 'True Positive']], columns=mat.columns, index=mat.index)
rubric_df

In [ ]:
r = rubric_df + ': ' + mat.values.astype(str)
r

In [ ]:
type(r)

In [ ]:
conf_m

In [ ]:
conf_m[0,0]

positive = survived

these calculate the rates
- tpr = tp/(tp+fn)
- fpr = fp/(fp+tn)
- tnr = tn/(tn+fp)
- fnr = fn/(fn+tp)

https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
#assign the values
tp = conf_m[1,1]
fp =conf_m[0,1] 
fn= conf_m[1,0]
tn =conf_m[0,0]

In [ ]:
#calculate the rate
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(tn+fp)
fnr = fn/(fn+tp)

In [ ]:
#     matrix = confusion_matrix(y, model.predict(X))
#     tpr = matrix[1,1] / (matrix[1,1] + matrix[1,0])
#     fpr = matrix[0,1] / (matrix[0,1] + matrix[0,0])
#     tnr = matrix[0,0] / (matrix[0,0] + matrix[0,1])
#     fnr = matrix[1,0] / (matrix[1,1] + matrix[1,0])

In [ ]:
#calculate the accuracy
acc = model1.score(X_train, y_train)
acc

In [ ]:
print(f'''
The accuracy for our model is {acc:.4}
The True Positive Rate is {tpr:.3}, The False Positive Rate is {fpr:.3},
The True Negative Rate is {tnr:.3}, and the False Negative Rate is {fnr:.3}
''')

In [ ]:
#classification report
clas_rep =pd.DataFrame(classification_report(y_train, prediction, output_dict=True)).T
clas_rep

In [ ]:
pd.Series(dict(zip(X_train.columns, model1.feature_importances_)))

**********************

## other models

In [ ]:
model11 = DecisionTreeClassifier(max_depth=3, max_features= 4)
model11.fit(X_train, y_train)
print(f'training score: {model11.score(X_train, y_train):.2%}')


In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model11,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
pd.Series(dict(zip(X_train.columns, model11.feature_importances_)))

In [ ]:

model2 = DecisionTreeClassifier(max_depth=3)
model2.fit(X_train, y_train)

print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
# we want traning score < validate score.
# an aceptable range is a diff of 5%

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model2,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
pd.Series(dict(zip(X_train.columns, model2.feature_importances_)))

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [ ]:
# ****************compute metrics on the TRAIN set

In [ ]:
#prediction based on our model
prediction = model.predict(X_train)

In [ ]:
prediction

In [ ]:
# check out the values in the predictions. we cannot use value_counts in array thats whay we conver it to pd.series
pd.Series(prediction).value_counts()

In [ ]:
#model Score: Accuracy

acc = model1.score(X_train, y_train)
acc

In [ ]:
train.head(1)

In [ ]:
#these are the actual numbers
train.survived.value_counts()

In [ ]:
#check if you change the order of labels

In [ ]:
confusion_matrix(y_train, prediction)

In [ ]:
confusion_matrix(y_train, prediction, labels=(1, 0) ) 

In [ ]:
confusion_matrix(y_train, prediction, labels=(0, 1) )#this is the default

In [ ]:
#the rows are the actual dead  because 0 + 307 = 307, 
confusion_matrix(y_train, prediction)

In [ ]:
cm= pd.DataFrame ((confusion_matrix(y_train, prediction )),index = ['actual_dead','actual_survived'], columns =['pred_dead','pred_survived' ])

In [ ]:
cm

In [ ]:
#model Score: Accuracy

acc = model1.score(X_train, y_train)
acc

In [ ]:
pd.DataFrame(classification_report(y_train, prediction, output_dict=True)).T

### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
#### on the validate set *****


- positive is : survived
- True Positive: predict survived and it is survived
- True Negative: predict death and it is death
- False Positive: predict survived and it is death
- False Negative: predict death and it is survived

In [ ]:
# print the model score on the validate set
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:
val_pred = model1.predict(X_validate)

In [ ]:
validate.survived.value_counts()

In [ ]:
confusion_matrix(validate.survived, val_pred)

In [ ]:
cm2= confusion_matrix(y_validate, val_pred)

In [ ]:
#just to see
pd.DataFrame (confusion_matrix(y_validate, val_pred),index = ['actual_death','actual_survived'], columns =['pred_death','pred_survived' ] )

In [ ]:
#psitive : survived
#assign the values
tp = cm2[1,1]
fp =cm2[0,1] 
fn= cm2[1,0]
tn =cm2[0,0]

In [ ]:
#calculate the rate
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(tn+fp)
fnr = fn/(fn+tp)

In [ ]:
print(f'''

The True Positive Rate is {tpr:.3}, The False Positive Rate is {fpr:.3},
The True Negative Rate is {tnr:.3}, and the False Negative Rate is {fnr:.3}
''')

In [ ]:
#accuracy
acc_model1 =accuracy_score(y_validate, val_pred)
acc_model1

In [ ]:
#precision
prec_model1= precision_score(y_validate, val_pred, pos_label = 1 )
prec_model1

In [ ]:
#recall
rec_model1= recall_score(y_validate, val_pred, pos_label = 1 )
rec_model1

In [ ]:
df = pd.DataFrame(classification_report(y_validate, val_pred, output_dict= True))
df

In [ ]:
df.rename(columns={'0': "dead", '1': "survived"}, inplace = True)

In [ ]:
df.T

### 5. Run through steps 2-4 using a different max_depth value.

In [ ]:

model2 = DecisionTreeClassifier(max_depth=2)
model2.fit(X_train, y_train)

print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
pd.Series(dict(zip(X_train.columns, model2.feature_importances_)))

In [ ]:
#using my function import model 
model.model_performs(X_train, y_train, model2)

In [ ]:
#if I want to see a dec_tree
model.dec_tree(model2, X_train)

In [ ]:
model3 = DecisionTreeClassifier(max_depth= 5)
model3.fit(X_train, y_train)

print(f'training score: {model3.score(X_train, y_train):.2%}')
print(f'validate score: {model3.score(X_validate, y_validate):.2%}')

In [ ]:
#using my function import model 
model.model_performs(X_train, y_train, model3)

In [ ]:
#if I want to see a dec_tree
model.dec_tree(model3, X_train)

In [ ]:
#accuracy
acc_model2 =accuracy_score(train.survived, train.prediction_2)

In [ ]:
#precision
prec_model2 =precision_score(train.survived, train.prediction_2, pos_label = 1 )

In [ ]:
#recall
rec_model2= recall_score(train.survived, train.prediction_2, pos_label = 1 )

### 6. Which model performs better on your in-sample data? (aka TRAIN set)

In [ ]:
# model1 using my func
model1 = DecisionTreeClassifier(max_depth=3, max_features=3)

model1.fit(X_train, y_train)

In [ ]:
model.model_performs(X_train, y_train, model1)

In [ ]:
#if I want to see a dec_tree
model.dec_tree(model1, X_train)

### 7.Which model performs best on your out-of-sample data, the validate set? (aka the VALIDATE set)

In [ ]:
# get predictions for our validation sets
val_pred_1 = model1.predict(X_validate)
val_pred_2 = model2.predict(X_validate)
val_pred_3 = model3.predict(X_validate)


In [ ]:
# get validation accuracy
acc_v_1 = model1.score(X_validate, y_validate)
acc_v_2 = model2.score(X_validate, y_validate)
acc_v_3 = model3.score(X_validate, y_validate)


In [ ]:
acc_v_1, acc_v_2, acc_v_3

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {model1.score(X_train, y_train):.2%}')
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:

# look at the model scores for training set and validate set
print(f'training score: {model1.score(X_train, y_train):.2%}')
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:

# look at the model scores for training set and validate set
print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {model3.score(X_train, y_train):.2%}')
print(f'validate score: {model3.score(X_validate, y_validate):.2%}')

In [ ]:
:
# 
# 
# We see here a very significant drop-off on model ,
# where we did not specify a maximum depth, 
# suggesting it was very overfit to the training set
# training score >>> validate score = overfit
# b
# 
#

In [ ]:
dot_data = export_graphviz(model1, feature_names= X_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data)

In [ ]:
graph.render('titanic_model_1_tree', view=True)

In [ ]:
dot_data = export_graphviz(model3, feature_names= X_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data)
graph.render('titanic_model_1_tree', view=True)

# Random Forest

## 1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [2]:
#acquire data
df = acquire.get_titanic_data()

In [3]:
#split data
train, validate, test = prepare.prep_titanic_data(df, column = 'age', method = 'median', dummies = ['embarked', 'sex'])


In [4]:
# drop out non-numerical columns or non-encoded version remaining in this data set.
#in this case, it is only 'passaner_id'
train.drop(columns = ['passenger_id'], inplace=True)
validate.drop(columns = ['passenger_id'], inplace=True)
test.drop(columns = ['passenger_id'], inplace=True)

In [5]:
# use the funnction that I create:

X_train, y_train = model.split_Xy(df = train, column = 'survived')
X_validate, y_validate = model.split_Xy(validate, 'survived') 
X_test, y_test = model.split_Xy(test, 'survived') 

In [6]:
# Establish our baseline.  The rate at which the assumption of the majority class matches the real values.  
#If a model does not perform better than this, it would not be wise to deploy.
baseline = (y_train.value_counts().idxmax() == y_train).mean()
baseline

survived    0.616466
dtype: float64

In [7]:
# create the Random Forest Model
rfmodel = RandomForestClassifier(min_samples_leaf=1, max_depth=10, random_state=1349)

In [8]:
# fit the thing
rfmodel.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=1349)

In [9]:
#this is the next step but I already have it in my function
#  use the thing
# y_pred = rfmodel.predict(X_train)

## 2. Evaluate your results using the model score, confusion matrix, and classification report.

In [10]:
model.model_performs(X_train, y_train, rfmodel)


    The accuracy for our model is 96.9880%

    The True Positive Rate is 92.147%,    The False Positive Rate is 0.000%,
    The True Negative Rate is 100.000%,    The False Negative Rate is 7.853%

    ________________________________________________________________________________
    

    The positive is  'survived'

    Confusion Matrix

                               pred_dead       pred_survived
actual_dead      True Negative: 307   False positive: 0
actual_survived  False Negative: 15  True Positive: 176


    ________________________________________________________________________________
    
    Classification Report:
    
                   precision    recall  f1-score    support
dead           0.953416  1.000000  0.976153  307.00000
survived       1.000000  0.921466  0.959128  191.00000
accuracy       0.969880  0.969880  0.969880    0.96988
macro avg      0.976708  0.960733  0.967640  498.00000
weighted avg   0.971283  0.969880  0.969623  498.00000


## 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [11]:
model.model_performs(X_train, y_train, rfmodel)


    The accuracy for our model is 96.9880%

    The True Positive Rate is 92.147%,    The False Positive Rate is 0.000%,
    The True Negative Rate is 100.000%,    The False Negative Rate is 7.853%

    ________________________________________________________________________________
    

    The positive is  'survived'

    Confusion Matrix

                               pred_dead       pred_survived
actual_dead      True Negative: 307   False positive: 0
actual_survived  False Negative: 15  True Positive: 176


    ________________________________________________________________________________
    
    Classification Report:
    
                   precision    recall  f1-score    support
dead           0.953416  1.000000  0.976153  307.00000
survived       1.000000  0.921466  0.959128  191.00000
accuracy       0.969880  0.969880  0.969880    0.96988
macro avg      0.976708  0.960733  0.967640  498.00000
weighted avg   0.971283  0.969880  0.969623  498.00000


## 4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.

In [12]:
# create the Random Forest Model
rfmod2 = RandomForestClassifier(min_samples_leaf=4, max_depth=4, random_state=1349)

# fit the model
rfmod2.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, min_samples_leaf=4, random_state=1349)

In [13]:
model.model_performs(X_train, y_train, rfmod2)


    The accuracy for our model is 85.1406%

    The True Positive Rate is 70.157%,    The False Positive Rate is 5.537%,
    The True Negative Rate is 94.463%,    The False Negative Rate is 29.843%

    ________________________________________________________________________________
    

    The positive is  'survived'

    Confusion Matrix

                               pred_dead       pred_survived
actual_dead      True Negative: 290  False positive: 17
actual_survived  False Negative: 57  True Positive: 134


    ________________________________________________________________________________
    
    Classification Report:
    
                   precision    recall  f1-score     support
dead           0.835735  0.944625  0.886850  307.000000
survived       0.887417  0.701571  0.783626  191.000000
accuracy       0.851406  0.851406  0.851406    0.851406
macro avg      0.861576  0.823098  0.835238  498.000000
weighted avg   0.855557  0.851406  0.847260  498.000000


## 5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

In [14]:
model.model_performs(X_validate, y_validate, rfmodel)


    The accuracy for our model is 82.2430%

    The True Positive Rate is 71.951%,    The False Positive Rate is 11.364%,
    The True Negative Rate is 88.636%,    The False Negative Rate is 28.049%

    ________________________________________________________________________________
    

    The positive is  'survived'

    Confusion Matrix

                               pred_dead       pred_survived
actual_dead      True Negative: 117  False positive: 15
actual_survived  False Negative: 23   True Positive: 59


    ________________________________________________________________________________
    
    Classification Report:
    
                   precision    recall  f1-score    support
dead           0.835714  0.886364  0.860294  132.00000
survived       0.797297  0.719512  0.756410   82.00000
accuracy       0.822430  0.822430  0.822430    0.82243
macro avg      0.816506  0.802938  0.808352  214.00000
weighted avg   0.820994  0.822430  0.820488  214.00000


model.model_performs(X_validate, y_validate, rfmod2)

## After making a few models, which one has the best performance (or closest metrics) on both train and validate?